In [1]:
import numpy as np
import pandas as pd
import csv

import os, os.path
import sys

# to move files from one directory to another
import shutil

In [2]:
param_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/"
non_expert_chosen_fields_dir = param_dir + "nonExpert_set2_fields/"

plots_dir = "/Users/hn/Documents/01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/"

In [3]:
chosen_fields = pd.read_csv(non_expert_chosen_fields_dir+"nonExpert_survey2_fields.csv")

print('There are [{ques_count}] questions.'.format(ques_count=chosen_fields.shape[0]))

There are [1429] questions.


In [4]:
chosen_fields.columns

Index(['ID', 'NDVI_TS_Name', 'corrected_RGB', 'TOA_RGB', 'latitude',
       'longitude', 'Question Text', 'CropTyp', 'Irrigtn', 'DataSrc', 'Acres',
       'ExctAcr', 'LstSrvD', 'county'],
      dtype='object')

In [5]:
bad_cols = ['corrected_RGB', 'TOA_RGB', 'Question Text']
chosen_fields.drop(labels=bad_cols, axis=1, inplace=True)

In [21]:
response_set_xl = pd.ExcelFile(non_expert_chosen_fields_dir + "nonExpert_survey2_responses.xlsx")
response_set_sheet_names = response_set_xl.sheet_names  # see all sheet names
# response_set_sheet_names = [x for x in response_set_sheet_names if 'Set 2' in x]
# response_set_sheet_names = sorted(response_set_sheet_names)
print (response_set_sheet_names)

['NE_S2_F1', 'NE_S2_F2', 'NE_S2_F3', 'NE_S2_F4', 'NE_S2_F5', 'NE_S2_F6', 'NE_S2_F7', 'NE_S2_F8', 'NE_S2_F9', 'NE_S2_F10', 'NE_S2_F11', 'NE_S2_F12', 'NE_S2_F13', 'NE_S2_F14', 'NE_S2_F15', 'NE_S2_F16', 'NE_S2_F17', 'NE_S2_F18', 'NE_S2_F19', 'NE_S2_F20', 'NE_S2_F21', 'NE_S2_F22', 'NE_S2_F23', 'NE_S2_F24']


# Read all the responses

Read and concatenate all the responses. Add the ID to each row as well.

In [22]:
# sample_response = response_set_xl.parse(response_set_sheet_names[3])
# sample_response = sample_response.drop(columns=['Timestamp'])
# sample_response
# no_questions = (sample_response.shape[1]-1) // 3
# no_questions

In [23]:
question_count = chosen_fields.shape[0]
question_count

1429

In [24]:
response_cols = ["ID", "Set", "Form", "Question",
                 "HosseinV", "KirtiV", "MichaelV", "SupriyaV",
                 "HosseinD", "KirtiD", "MichaelD", "SupriyaD",
                 "HosseinC", "KirtiC", "MichaelC", "SupriyaC"]

all_responses = pd.DataFrame(columns=response_cols, 
                             index=range(question_count))


row_number = -1
for response_sheet_name in response_set_sheet_names:
    sample_response = response_set_xl.parse(response_sheet_name)
    sample_response = sample_response.drop(columns=['Timestamp'])
    number_of_questions = (sample_response.shape[1]-1)//3
    
    for question_number in range(1, number_of_questions+1):
        # list(curr_tbl.columns)[1].split("(")[1].split(")")[0]
        row_number+=1
        
        # Pick columns corresponding to current question!
        col_start = (question_number*3)-2
        curr_tbl = sample_response.iloc[:, [0, col_start, col_start+1, col_start+2]].copy()

        #  enter the Set number
        ID = curr_tbl.columns[1].split("(")[1].split(")")[0]
        all_responses.loc[row_number, "ID"]=ID
        all_responses.loc[row_number, "Set"]=response_sheet_name.split("_")[1]
                 #response_sheet_name.split("_")[2]
        all_responses.loc[row_number, "Form"]=int(response_sheet_name.split("_")[2][1:]) 
        all_responses.loc[row_number, "Question"]=curr_tbl.columns[1].split(" ")[1]

        for email in sample_response["Email Address"].values:
            if "noorazar" in email:
                # Hossein
                all_responses.loc[row_number, "HosseinV"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][1]
                all_responses.loc[row_number, "HosseinD"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][2]
                all_responses.loc[row_number, "HosseinC"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][3]
            elif "brady" in email:
                # Michael
                all_responses.loc[row_number, "MichaelV"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][1]
                all_responses.loc[row_number, "MichaelD"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][2]
                all_responses.loc[row_number, "MichaelC"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][3]
                
            elif "kirti" in email:
                # Kirti
                all_responses.loc[row_number, "KirtiV"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][1]
                all_responses.loc[row_number, "KirtiD"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][2]
                all_responses.loc[row_number, "KirtiC"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][3]
                
            elif "sup" in email:
                # Supriya
                # print (curr_tbl[curr_tbl["Email Address"]==email].values[0][1])
                all_responses.loc[row_number, "SupriyaV"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][1]
                all_responses.loc[row_number, "SupriyaD"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][2]
                all_responses.loc[row_number, "SupriyaC"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][3]

all_responses.head(2)

,ID,Set,Form,Question,HosseinV,KirtiV,MichaelV,SupriyaV,HosseinD,KirtiD,MichaelD,SupriyaD,HosseinC,KirtiC,MichaelC,SupriyaC
0,101534_WSDA_SF_2018,S2,1,1,Single,Cover,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,108118_WSDA_SF_2017,S2,1,2,Single,Single,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
all_responses.tail(2)

,ID,Set,Form,Question,HosseinV,KirtiV,MichaelV,SupriyaV,HosseinD,KirtiD,MichaelD,SupriyaD,HosseinC,KirtiC,MichaelC,SupriyaC
1427,54907_WSDA_SF_2016,S2,24,48,Double/Mustard,NaN,Double,Double,NaN,NaN,Check for Yes,NaN,NaN,NaN,NaN,NaN
1428,95998_WSDA_SF_2018,S2,24,49,Single,NaN,Single,Single,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
out_name = non_expert_chosen_fields_dir + "NE_set2_all_responses.csv"
all_responses.to_csv(out_name, index = False)

In [27]:
chosen_fields.head(2)

,ID,NDVI_TS_Name,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,101534_WSDA_SF_2018,FranklinYakima2018_101534_WSDA_SF_2018.png,46.321880,-119.050357,barley,center pivot,wsda,3,3.220729,2018/08/14 00:00:00,Franklin
1,108118_WSDA_SF_2017,Grant2017_108118_WSDA_SF_2017.png,46.872243,-119.410143,barley,rill,wsda,5,4.956565,2017/07/12,Grant


In [28]:
all_responses = pd.merge(all_responses, chosen_fields, on=['ID'], how='left')
all_responses = all_responses.fillna("No")
all_responses.head(2)

,ID,Set,Form,Question,HosseinV,KirtiV,MichaelV,SupriyaV,HosseinD,KirtiD,...,NDVI_TS_Name,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,101534_WSDA_SF_2018,S2,1,1,Single,Cover,No,No,No,No,...,FranklinYakima2018_101534_WSDA_SF_2018.png,46.321880,-119.050357,barley,center pivot,wsda,3,3.220729,2018/08/14 00:00:00,Franklin
1,108118_WSDA_SF_2017,S2,1,2,Single,Single,No,No,No,No,...,Grant2017_108118_WSDA_SF_2017.png,46.872243,-119.410143,barley,rill,wsda,5,4.956565,2017/07/12,Grant


# Clean up the all_responses
Keep only fields that there is a disagreement, or discussion is requested, or there is a comment.
In other word, toss the ones that are good.

In [31]:
print (all_responses.shape)
all_responses.head(2)

(1429, 26)


,ID,Set,Form,Question,HosseinV,KirtiV,MichaelV,SupriyaV,HosseinD,KirtiD,...,NDVI_TS_Name,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,101534_WSDA_SF_2018,S2,1,1,Single,Cover,No,No,No,No,...,FranklinYakima2018_101534_WSDA_SF_2018.png,46.321880,-119.050357,barley,center pivot,wsda,3,3.220729,2018/08/14 00:00:00,Franklin
1,108118_WSDA_SF_2017,S2,1,2,Single,Single,No,No,No,No,...,Grant2017_108118_WSDA_SF_2017.png,46.872243,-119.410143,barley,rill,wsda,5,4.956565,2017/07/12,Grant


### Create a long PDF of fields where discussion is requested

In [32]:
import subprocess
os.getcwd()

'/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/NonExpert_Analysis'

# Set 13-24:

In [33]:
all_responses_backUp=all_responses.copy()

In [34]:
all_responses.drop(labels=["KirtiV", "KirtiD", "KirtiC"], axis='columns', inplace=True)
all_responses = all_responses[all_responses['Form']>=13]

In [39]:
os.chdir(non_expert_chosen_fields_dir + 'NE_Set2_PDFs/')
disagreement_count = 0
discuss_count = 0
comment_count = 0

with open("NE_Set_2_3in1_Forms13_24_NoKirti_MustardCoverPresent.tex","w") as file:
    file.write("\\documentclass{article}\n")

    file.write("\\usepackage[osf,sc]{mathpazo}\n")
    file.write("\\usepackage{color}\n")
    file.write("\\definecolor{mgreen}{RGB}{25,147,100}\n")

    file.write("\\usepackage{mathtools,hyperref}\n")
    file.write("\\hypersetup{\n")
    file.write("      colorlinks=true,\n")
    file.write("      linkcolor=cyan,\n")
    file.write("      filecolor=cyan,\n")
    file.write("      urlcolor=mgreen,\n")
    file.write("      citecolor=cyan}\n")
    

    file.write("\\usepackage[margin=1in]{geometry}\n")
    file.write("\\usepackage[T1]{fontenc}\n")
    
    file.write("\\usepackage{caption}\n")
    file.write("\\captionsetup{justification= raggedright, singlelinecheck = false}\n")

    file.write("\\title{\\bf Set 2 - Discussion, Comments, and Disgreements}\n")
    file.write("\\author{}\n")
    file.write("\\date{}\n")


    file.write("\\begin{document}\n")
    file.write("\\maketitle\n")

    # file.write("\\section{Discussion, Comments, and Disgreements}\n")
    
    file.write("\n")
    file.write("\n")
    
    for a_row in all_responses.index:
        curr_row = all_responses.loc[a_row]
        
        ##### Disagreement
        if len(np.unique(curr_row[["HosseinV", "MichaelV", "SupriyaV"]].values))>1: # KirtiV
            is_there_disagreement=True
            disagreement_count+=1
        else:
            is_there_disagreement=False
        
        ##### Discussion
        if str(curr_row["HosseinD"])==str(curr_row["MichaelD"])==str(curr_row["SupriyaD"])=='No':
            discuss_or_not=False
        else:
            discuss_or_not=True
            discuss_count+=1
        
        ##### Comment
        if str(curr_row["HosseinC"])==str(curr_row["MichaelC"])==str(curr_row["SupriyaC"])=='No':
            comment_or_not=False
        else:
            comment_or_not=True
            comment_count+=1
        
        if is_there_disagreement or discuss_or_not or comment_or_not:
            TS_file_name = curr_row["NDVI_TS_Name"]
            file.write("\\begin{figure*}[ht]\n")
            file.write("\\centering\n")
            file.write("\\includegraphics[width=1\\textwidth]{/Users/hn/Documents/" + \
                         "01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/" + \
                             TS_file_name + "}\n")
            file.write("\\caption[]{" + \
                       "\\textbf{\\color{red}{" + curr_row["CropTyp"] + "}}" + \
                       ", ID: " + curr_row["ID"].replace("_", "\_") + \
                       ", Form: " + str(curr_row["Form"]) + \
                       ", Question: " + curr_row["Question"] + "." + \
                       "}\n")
            file.write("\\label{fig:figure" + str(a_row) + "}\n")
            file.write("\\end{figure*}\n")
            file.write("\n")
        
            if discuss_or_not:
                file.write("\\noindent \\textbf{Discuss? {\color{red}{Yes}}}" + "\\\\\n")
            else:
                file.write("\\noindent \\textbf{Discuss? {\color{mgreen}{No}}}" + "\\\\\n")
            
            # file.write("\\noindent Kirti: " + str(curr_row["KirtiD"]) + "\\\\\n")
            file.write("\\noindent Michael: ~" + str(curr_row["MichaelD"]) + "\\\\\n")
            file.write("\\noindent Supriya: ~" + str(curr_row["SupriyaD"]) + "\\\\\n")
            file.write("\\noindent Hossein: ~" + str(curr_row["HosseinD"]) + "\\\\\n")

            file.write("\n")
            file.write("\n")
            if is_there_disagreement:
                file.write("\\noindent \\textbf{Disagreement? {\color{red}{Yes}}}" + "\\\\\n")
#                 file.write("\\noindent Perry's Vote:~~~~ " + curr_row["PerryV"] + "\\\\\n")
#                 file.write("Andrew's Vote: " + str(curr_row["AndrewV"]) + "\\\\\n")
            else:
                file.write("\\noindent \\textbf{Disagreement? {\color{mgreen}{No}}}" + "\\\\\n")
#               file.write("\\noindent Perry's Vote:~~~~ " + curr_row["PerryV"] + "\\\\\n")
#               file.write("Andrew's Vote: " + curr_row['AndrewV'] + "\\\\\n")
            file.write("\\noindent Michael's Vote: " + curr_row['MichaelV'] + "\\\\\n")
            # file.write("Kirti's Vote:~~~~ " + str(curr_row["KirtiV"]) + "\\\\\n")            
            file.write("Supriya's Vote: " + curr_row['SupriyaV'] + "\\\\\n")
            file.write("Hossein's Vote: " + curr_row['HosseinV'] + "\\\\\n")            

            file.write("\n")
            file.write("\n")
            file.write("\\noindent \\textbf{Comments}" + "\\\\\n")
            file.write("\\noindent  Michael: " + str(curr_row["MichaelC"]) + "\\\\\n")
            # file.write("Kirti: ~~~~" + str(curr_row["KirtiC"]) + "\\\\\n")
            file.write("Supriya: " + str(curr_row["SupriyaC"]) + "\\\\\n")
            file.write("Hossein: " + str(curr_row["HosseinC"]) + "\\\\\n")

            file.write("\\clearpage\n")

    file.write("\\noindent Disagreement count: " + str(disagreement_count) + "\\\\\n")
    file.write("Discussion count: " + str(discuss_count) + "\\\\\n")
    file.write("Comment count: " + str(comment_count) + "\\\\\n")
    
    file.write("\\end{document}\n")
    
print (disagreement_count)
print (discuss_count)
print (comment_count)
all_responses.shape

123
25
0


(709, 23)

In [40]:
print (all_responses['MichaelV'].unique())
print (all_responses['SupriyaV'].unique())
print (all_responses['HosseinV'].unique())

['Single' 'Double' 'Unsure']
['Single' 'Double' 'Double/Mustard']
['Single' 'Double' 'Cover' 'Unsure' 'Double/Mustard']


# Replace 
  - Double/Mustard with Double
  - Cover with Single

In [55]:
all_responses.loc[all_responses['HosseinV']=="Double/Mustard", 'HosseinV']="Double"
all_responses.loc[all_responses['SupriyaV']=="Double/Mustard", 'SupriyaV']="Double"
all_responses.loc[all_responses['MichaelV']=="Double/Mustard", 'MichaelV']="Double"
# all_responses.loc[all_responses['KirtiV']=="Double/Mustard", 'KirtiV']="Double"

all_responses.loc[all_responses['HosseinV']=="Cover", 'HosseinV']="Single"
all_responses.loc[all_responses['SupriyaV']=="Cover", 'SupriyaV']="Single"
all_responses.loc[all_responses['MichaelV']=="Cover", 'MichaelV']="Single"
# all_responses.loc[all_responses['KirtiV']=="Cover", 'KirtiV']="Single"
all_responses['HosseinV'].unique()

array(['Single', 'Double', 'Unsure'], dtype=object)

In [56]:
out_name = non_expert_chosen_fields_dir + "NE_set2_Forms13_24_NoCoverMustard.csv"
all_responses.to_csv(out_name, index = False)

In [57]:
os.chdir(non_expert_chosen_fields_dir + 'NE_Set2_PDFs/')
disagreement_count = 0
discuss_count = 0
comment_count = 0

with open("NE_Set_2_3in1_Forms13_24_NoKirti_MustardCoverConverted.tex","w") as file:
    file.write("\\documentclass{article}\n")

    file.write("\\usepackage[osf,sc]{mathpazo}\n")
    file.write("\\usepackage{color}\n")
    file.write("\\definecolor{mgreen}{RGB}{25,147,100}\n")

    file.write("\\usepackage{mathtools,hyperref}\n")
    file.write("\\hypersetup{\n")
    file.write("      colorlinks=true,\n")
    file.write("      linkcolor=cyan,\n")
    file.write("      filecolor=cyan,\n")
    file.write("      urlcolor=mgreen,\n")
    file.write("      citecolor=cyan}\n")
    

    file.write("\\usepackage[margin=1in]{geometry}\n")
    file.write("\\usepackage[T1]{fontenc}\n")
    
    file.write("\\usepackage{caption}\n")
    file.write("\\captionsetup{justification= raggedright, singlelinecheck = false}\n")

    file.write("\\title{\\bf Set 2 - Discussion, Comments, and Disgreements}\n")
    file.write("\\author{}\n")
    file.write("\\date{}\n")


    file.write("\\begin{document}\n")
    file.write("\\maketitle\n")

    # file.write("\\section{Discussion, Comments, and Disgreements}\n")
    
    file.write("\n")
    file.write("\n")
    
    for a_row in all_responses.index:
        curr_row = all_responses.loc[a_row]
        
        ##### Disagreement
        if len(np.unique(curr_row[["HosseinV", "MichaelV", "SupriyaV"]].values))>1: # KirtiV
            is_there_disagreement=True
            disagreement_count+=1
        else:
            is_there_disagreement=False
        
        ##### Discussion
        if str(curr_row["HosseinD"])==str(curr_row["MichaelD"])==str(curr_row["SupriyaD"])=='No':
            discuss_or_not=False
        else:
            discuss_or_not=True
            discuss_count+=1
        
        ##### Comment
        if str(curr_row["HosseinC"])==str(curr_row["MichaelC"])==str(curr_row["SupriyaC"])=='No':
            comment_or_not=False
        else:
            comment_or_not=True
            comment_count+=1
        
        if is_there_disagreement or discuss_or_not or comment_or_not:
            TS_file_name = curr_row["NDVI_TS_Name"]
            file.write("\\begin{figure*}[ht]\n")
            file.write("\\centering\n")
            file.write("\\includegraphics[width=1\\textwidth]{/Users/hn/Documents/" + \
                         "01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/" + \
                             TS_file_name + "}\n")
            file.write("\\caption[]{" + \
                       "\\textbf{\\color{red}{" + curr_row["CropTyp"] + "}}" + \
                       ", ID: " + curr_row["ID"].replace("_", "\_") + \
                       ", Form: " + str(curr_row["Form"]) + \
                       ", Question: " + curr_row["Question"] + "." + \
                       "}\n")
            file.write("\\label{fig:figure" + str(a_row) + "}\n")
            file.write("\\end{figure*}\n")
            file.write("\n")
        
            if discuss_or_not:
                file.write("\\noindent \\textbf{Discuss? {\color{red}{Yes}}}" + "\\\\\n")
            else:
                file.write("\\noindent \\textbf{Discuss? {\color{mgreen}{No}}}" + "\\\\\n")
            
            # file.write("\\noindent Kirti: " + str(curr_row["KirtiD"]) + "\\\\\n")
            file.write("\\noindent Michael: ~" + str(curr_row["MichaelD"]) + "\\\\\n")
            file.write("\\noindent Supriya: ~" + str(curr_row["SupriyaD"]) + "\\\\\n")
            file.write("\\noindent Hossein: ~" + str(curr_row["HosseinD"]) + "\\\\\n")

            file.write("\n")
            file.write("\n")
            if is_there_disagreement:
                file.write("\\noindent \\textbf{Disagreement? {\color{red}{Yes}}}" + "\\\\\n")
#                 file.write("\\noindent Perry's Vote:~~~~ " + curr_row["PerryV"] + "\\\\\n")
#                 file.write("Andrew's Vote: " + str(curr_row["AndrewV"]) + "\\\\\n")
            else:
                file.write("\\noindent \\textbf{Disagreement? {\color{mgreen}{No}}}" + "\\\\\n")
#               file.write("\\noindent Perry's Vote:~~~~ " + curr_row["PerryV"] + "\\\\\n")
#               file.write("Andrew's Vote: " + curr_row['AndrewV'] + "\\\\\n")
            file.write("\\noindent Michael's Vote: " + curr_row['MichaelV'] + "\\\\\n")
            # file.write("Kirti's Vote:~~~~ " + str(curr_row["KirtiV"]) + "\\\\\n")            
            file.write("Supriya's Vote: " + curr_row['SupriyaV'] + "\\\\\n")
            file.write("Hossein's Vote: " + curr_row['HosseinV'] + "\\\\\n")            

            file.write("\n")
            file.write("\n")
            file.write("\\noindent \\textbf{Comments}" + "\\\\\n")
            file.write("\\noindent  Michael: " + str(curr_row["MichaelC"]) + "\\\\\n")
            # file.write("Kirti: ~~~~" + str(curr_row["KirtiC"]) + "\\\\\n")
            file.write("Supriya: " + str(curr_row["SupriyaC"]) + "\\\\\n")
            file.write("Hossein: " + str(curr_row["HosseinC"]) + "\\\\\n")

            file.write("\\clearpage\n")

    file.write("\\noindent Disagreement count: " + str(disagreement_count) + "\\\\\n")
    file.write("Discussion count: " + str(discuss_count) + "\\\\\n")
    file.write("Comment count: " + str(comment_count) + "\\\\\n")
    
    file.write("\\end{document}\n")
    
print (disagreement_count)
print (discuss_count)
print (comment_count)
all_responses.shape

52
25
0


(709, 23)

# Set 1-12. KR and HN

In [68]:
all_responses=all_responses_backUp.copy()

In [69]:
all_responses = all_responses[all_responses['Form']<=12]
all_responses.head(2)

,ID,Set,Form,Question,HosseinV,KirtiV,MichaelV,SupriyaV,HosseinD,KirtiD,...,NDVI_TS_Name,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,101534_WSDA_SF_2018,S2,1,1,Single,Cover,No,No,No,No,...,FranklinYakima2018_101534_WSDA_SF_2018.png,46.321880,-119.050357,barley,center pivot,wsda,3,3.220729,2018/08/14 00:00:00,Franklin
1,108118_WSDA_SF_2017,S2,1,2,Single,Single,No,No,No,No,...,Grant2017_108118_WSDA_SF_2017.png,46.872243,-119.410143,barley,rill,wsda,5,4.956565,2017/07/12,Grant


In [70]:
all_responses.drop(labels=["MichaelV", "MichaelD", "MichaelC", 
                           "SupriyaV", "SupriyaD", "SupriyaC"],
                   axis='columns', inplace=True)

In [71]:
all_responses.head(2)

,ID,Set,Form,Question,HosseinV,KirtiV,HosseinD,KirtiD,HosseinC,KirtiC,NDVI_TS_Name,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,101534_WSDA_SF_2018,S2,1,1,Single,Cover,No,No,No,No,FranklinYakima2018_101534_WSDA_SF_2018.png,46.321880,-119.050357,barley,center pivot,wsda,3,3.220729,2018/08/14 00:00:00,Franklin
1,108118_WSDA_SF_2017,S2,1,2,Single,Single,No,No,No,No,Grant2017_108118_WSDA_SF_2017.png,46.872243,-119.410143,barley,rill,wsda,5,4.956565,2017/07/12,Grant


# Write all votes separately, i.e. cover is different from single

In [72]:
os.chdir(non_expert_chosen_fields_dir + 'NE_Set2_PDFs/')

disagreement_count = 0
discuss_count = 0
comment_count = 0

with open("NE_Set_2_3in1_Forms1_12_NoMikeSuprita_MustardCoverPresent.tex","w") as file:
    file.write("\\documentclass{article}\n")

    file.write("\\usepackage[osf,sc]{mathpazo}\n")
    file.write("\\usepackage{color}\n")
    file.write("\\definecolor{mgreen}{RGB}{25,147,100}\n")

    file.write("\\usepackage{mathtools,hyperref}\n")
    file.write("\\hypersetup{\n")
    file.write("      colorlinks=true,\n")
    file.write("      linkcolor=cyan,\n")
    file.write("      filecolor=cyan,\n")
    file.write("      urlcolor=mgreen,\n")
    file.write("      citecolor=cyan}\n")
    

    file.write("\\usepackage[margin=1in]{geometry}\n")
    file.write("\\usepackage[T1]{fontenc}\n")
    
    file.write("\\usepackage{caption}\n")
    file.write("\\captionsetup{justification= raggedright, singlelinecheck = false}\n")

    file.write("\\title{\\bf Set 2 - Discussion, Comments, and Disgreements}\n")
    file.write("\\author{}\n")
    file.write("\\date{}\n")


    file.write("\\begin{document}\n")
    file.write("\\maketitle\n")

    # file.write("\\section{Discussion, Comments, and Disgreements}\n")
    
    file.write("\n")
    file.write("\n")
    
    for a_row in all_responses.index:
        curr_row = all_responses.loc[a_row]
        
        ##### Disagreement
        if len(np.unique(curr_row[["HosseinV", "KirtiV"]].values))>1: # KirtiV
            is_there_disagreement=True
            disagreement_count+=1
        else:
            is_there_disagreement=False
        
        ##### Discussion
        if str(curr_row["HosseinD"])==str(curr_row["KirtiD"])=='No':
            discuss_or_not=False
        else:
            discuss_or_not=True
            discuss_count+=1
        
        ##### Comment
        if str(curr_row["HosseinC"])==str(curr_row["KirtiC"])=='No':
            comment_or_not=False
        else:
            comment_or_not=True
            comment_count+=1
        
        if is_there_disagreement or discuss_or_not or comment_or_not:
            TS_file_name = curr_row["NDVI_TS_Name"]
            file.write("\\begin{figure*}[ht]\n")
            file.write("\\centering\n")
            file.write("\\includegraphics[width=1\\textwidth]{/Users/hn/Documents/" + \
                         "01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/" + \
                             TS_file_name + "}\n")
            file.write("\\caption[]{" + \
                       "\\textbf{\\color{red}{" + curr_row["CropTyp"] + "}}" + \
                       ", ID: " + curr_row["ID"].replace("_", "\_") + \
                       ", Form: " + str(curr_row["Form"]) + \
                       ", Question: " + curr_row["Question"] + "." + \
                       "}\n")
            file.write("\\label{fig:figure" + str(a_row) + "}\n")
            file.write("\\end{figure*}\n")
            file.write("\n")
        
            if discuss_or_not:
                file.write("\\noindent \\textbf{Discuss? {\color{red}{Yes}}}" + "\\\\\n")
            else:
                file.write("\\noindent \\textbf{Discuss? {\color{mgreen}{No}}}" + "\\\\\n")
            
            # file.write("\\noindent Kirti: " + str(curr_row["KirtiD"]) + "\\\\\n")
            file.write("\\noindent Kirti: ~" + str(curr_row["KirtiD"]) + "\\\\\n")
            file.write("\\noindent Hossein: ~" + str(curr_row["HosseinD"]) + "\\\\\n")

            file.write("\n")
            file.write("\n")
            if is_there_disagreement:
                file.write("\\noindent \\textbf{Disagreement? {\color{red}{Yes}}}" + "\\\\\n")
#                 file.write("\\noindent Perry's Vote:~~~~ " + curr_row["PerryV"] + "\\\\\n")
#                 file.write("Andrew's Vote: " + str(curr_row["AndrewV"]) + "\\\\\n")
            else:
                file.write("\\noindent \\textbf{Disagreement? {\color{mgreen}{No}}}" + "\\\\\n")

            file.write("Kirti's Vote: " + curr_row['KirtiV'] + "\\\\\n")
            file.write("Hossein's Vote: " + curr_row['HosseinV'] + "\\\\\n")            

            file.write("\n")
            file.write("\n")
            file.write("\\noindent \\textbf{Comments}" + "\\\\\n")
            file.write("\\noindent Kirti: " + str(curr_row["KirtiC"]) + "\\\\\n")
            file.write("Hossein: " + str(curr_row["HosseinC"]) + "\\\\\n")

            file.write("\\clearpage\n")

    file.write("\\noindent Disagreement count: " + str(disagreement_count) + "\\\\\n")
    file.write("Discussion count: " + str(discuss_count) + "\\\\\n")
    file.write("Comment count: " + str(comment_count) + "\\\\\n")
    
    file.write("\\end{document}\n")
    
print (disagreement_count)
print (discuss_count)
print (comment_count)
all_responses.shape

85
2
38


(720, 20)

# Replace Cover with Single.

In [73]:
all_responses.loc[all_responses['HosseinV']=="Double/Mustard", 'HosseinV']="Double"
# all_responses.loc[all_responses['SupriyaV']=="Double/Mustard", 'SupriyaV']="Double"
# all_responses.loc[all_responses['MichaelV']=="Double/Mustard", 'MichaelV']="Double"
all_responses.loc[all_responses['KirtiV']=="Double/Mustard", 'KirtiV']="Double"

all_responses.loc[all_responses['HosseinV']=="Cover", 'HosseinV']="Single"
# all_responses.loc[all_responses['SupriyaV']=="Cover", 'SupriyaV']="Single"
# all_responses.loc[all_responses['MichaelV']=="Cover", 'MichaelV']="Single"
all_responses.loc[all_responses['KirtiV']=="Cover", 'KirtiV']="Single"
print (all_responses['HosseinV'].unique())

out_name = non_expert_chosen_fields_dir + "NE_set2_Forms1_12_NoCoverMustard.csv"
all_responses.to_csv(out_name, index = False)

['Single' 'Double' 'Unsure']


In [74]:
os.chdir(non_expert_chosen_fields_dir + 'NE_Set2_PDFs/')
disagreement_count = 0
discuss_count = 0
comment_count = 0

with open("NE_Set_2_3in1_Forms1_12_NoMichaelSupriya_MustardCoverConverted.tex","w") as file:
    file.write("\\documentclass{article}\n")

    file.write("\\usepackage[osf,sc]{mathpazo}\n")
    file.write("\\usepackage{color}\n")
    file.write("\\definecolor{mgreen}{RGB}{25,147,100}\n")

    file.write("\\usepackage{mathtools,hyperref}\n")
    file.write("\\hypersetup{\n")
    file.write("      colorlinks=true,\n")
    file.write("      linkcolor=cyan,\n")
    file.write("      filecolor=cyan,\n")
    file.write("      urlcolor=mgreen,\n")
    file.write("      citecolor=cyan}\n")
    

    file.write("\\usepackage[margin=1in]{geometry}\n")
    file.write("\\usepackage[T1]{fontenc}\n")
    
    file.write("\\usepackage{caption}\n")
    file.write("\\captionsetup{justification= raggedright, singlelinecheck = false}\n")

    file.write("\\title{\\bf Set 2 - Discussion, Comments, and Disgreements}\n")
    file.write("\\author{}\n")
    file.write("\\date{}\n")

    file.write("\\begin{document}\n")
    file.write("\\maketitle\n")

    # file.write("\\section{Discussion, Comments, and Disgreements}\n")
    
    file.write("\n")
    file.write("\n")
    
    for a_row in all_responses.index:
        curr_row = all_responses.loc[a_row]
        
        ##### Disagreement
        if len(np.unique(curr_row[["HosseinV", "KirtiV"]].values))>1: # KirtiV
            is_there_disagreement=True
            disagreement_count+=1
        else:
            is_there_disagreement=False
        
        ##### Discussion
        if str(curr_row["HosseinD"])==str(curr_row["KirtiD"])=='No':
            discuss_or_not=False
        else:
            discuss_or_not=True
            discuss_count+=1
        
        ##### Comment
        if str(curr_row["HosseinC"])==str(curr_row["KirtiC"])=='No':
            comment_or_not=False
        else:
            comment_or_not=True
            comment_count+=1
        
        if is_there_disagreement or discuss_or_not or comment_or_not:
            TS_file_name = curr_row["NDVI_TS_Name"]
            file.write("\\begin{figure*}[ht]\n")
            file.write("\\centering\n")
            file.write("\\includegraphics[width=1\\textwidth]{/Users/hn/Documents/" + \
                         "01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/" + \
                             TS_file_name + "}\n")
            file.write("\\caption[]{" + \
                       "\\textbf{\\color{red}{" + curr_row["CropTyp"] + "}}" + \
                       ", ID: " + curr_row["ID"].replace("_", "\_") + \
                       ", Form: " + str(curr_row["Form"]) + \
                       ", Question: " + curr_row["Question"] + "." + \
                       "}\n")
            file.write("\\label{fig:figure" + str(a_row) + "}\n")
            file.write("\\end{figure*}\n")
            file.write("\n")
        
            if discuss_or_not:
                file.write("\\noindent \\textbf{Discuss? {\color{red}{Yes}}}" + "\\\\\n")
            else:
                file.write("\\noindent \\textbf{Discuss? {\color{mgreen}{No}}}" + "\\\\\n")
            
            file.write("\\noindent Kirti: " + str(curr_row["KirtiD"]) + "\\\\\n")
            # file.write("\\noindent Michael: ~" + str(curr_row["MichaelD"]) + "\\\\\n")
            # file.write("\\noindent Supriya: ~" + str(curr_row["SupriyaD"]) + "\\\\\n")
            file.write("\\noindent Hossein: ~" + str(curr_row["HosseinD"]) + "\\\\\n")

            file.write("\n")
            file.write("\n")
            if is_there_disagreement:
                file.write("\\noindent \\textbf{Disagreement? {\color{red}{Yes}}}" + "\\\\\n")
            else:
                file.write("\\noindent \\textbf{Disagreement? {\color{mgreen}{No}}}" + "\\\\\n")

#            file.write("\\noindent Michael's Vote: " + curr_row['MichaelV'] + "\\\\\n")
            file.write("\\noindent Kirti's Vote:~~~~ " + str(curr_row["KirtiV"]) + "\\\\\n")            
#             file.write("Supriya's Vote: " + curr_row['SupriyaV'] + "\\\\\n")
            file.write("Hossein's Vote: " + curr_row['HosseinV'] + "\\\\\n")            

            file.write("\n")
            file.write("\n")
            file.write("\\noindent \\textbf{Comments}" + "\\\\\n")
            # file.write("\\noindent  Michael: " + str(curr_row["MichaelC"]) + "\\\\\n")
            file.write("\\noindent Kirti: ~~~~" + str(curr_row["KirtiC"]) + "\\\\\n")
            # file.write("Supriya: " + str(curr_row["SupriyaC"]) + "\\\\\n")
            file.write("Hossein: " + str(curr_row["HosseinC"]) + "\\\\\n")

            file.write("\\clearpage\n")

    file.write("\\noindent Disagreement count: " + str(disagreement_count) + "\\\\\n")
    file.write("Discussion count: " + str(discuss_count) + "\\\\\n")
    file.write("Comment count: " + str(comment_count) + "\\\\\n")
    
    file.write("\\end{document}\n")
    
print (disagreement_count)
print (discuss_count)
print (comment_count)
all_responses.shape

54
2
38


(720, 20)

# Pre-meeting Agreements and Disagreements

In [102]:
all_responses = all_responses_backUp.copy()

In [103]:
all_responses.loc[all_responses['HosseinV']=="Double/Mustard", 'HosseinV']="Double"
all_responses.loc[all_responses['SupriyaV']=="Double/Mustard", 'SupriyaV']="Double"
all_responses.loc[all_responses['MichaelV']=="Double/Mustard", 'MichaelV']="Double"
all_responses.loc[all_responses['KirtiV']=="Double/Mustard", 'KirtiV']="Double"

all_responses.loc[all_responses['HosseinV']=="Cover", 'HosseinV']="Single"
all_responses.loc[all_responses['SupriyaV']=="Cover", 'SupriyaV']="Single"
all_responses.loc[all_responses['MichaelV']=="Cover", 'MichaelV']="Single"
all_responses.loc[all_responses['KirtiV']=="Cover", 'KirtiV']="Single"
all_responses['HosseinV'].unique()

array(['Single', 'Double', 'Unsure'], dtype=object)

In [104]:
responses_1_12 = all_responses[all_responses.Form<=12].copy()
responses_13_24 = all_responses[all_responses.Form>=13].copy()

In [105]:
responses_1_12_agreement = responses_1_12[responses_1_12.HosseinV==responses_1_12.KirtiV]
responses_1_12_disagreement = responses_1_12[responses_1_12.HosseinV!=responses_1_12.KirtiV]

In [119]:
responses_13_24_agreement = responses_13_24[responses_13_24.HosseinV==responses_13_24.MichaelV]
responses_13_24_agreement = responses_13_24_agreement[responses_13_24_agreement.HosseinV==\
                                                      responses_13_24_agreement.SupriyaV]

responses_13_24_disagreement = responses_13_24[~responses_13_24.ID.isin(list(responses_13_24_agreement.ID))]

### Disagreement IDs

In [ ]:
L13 = list(responses_13_24_disagreement.ID.values)
L1 = list(responses_1_12_disagreement.ID.values)

In [143]:
L13 = list(responses_13_24_disagreement.ID.values)
L1 = list(responses_1_12_disagreement.ID.values)
disagreement_IDs = pd.DataFrame(L1+L13,  columns=["disagreement_IDs"])

In [144]:
out_name = non_expert_chosen_fields_dir + "NE_set2_disagreement_IDs.csv"
disagreement_IDs.to_csv(out_name, index = False)

### PreMeeting Consensus IDs

In [145]:
NE_premeeting_consensus = pd.concat([responses_1_12_agreement, responses_13_24_agreement])
out_name = non_expert_chosen_fields_dir + "NE_set2_premeeting_Consensus.csv"
NE_premeeting_consensus.to_csv(out_name, index = False)

In [149]:
out_name = non_expert_chosen_fields_dir + "NE_set2_premeeting_Consensus_IDs_Votes.csv"
NE_premeeting_consensus[["ID", "HosseinV"]].to_csv(out_name, index = False)

# DAMN, there are some agreements on "Unsure!"

In [158]:
unsureAgreement = responses_1_12_agreement[responses_1_12_agreement.HosseinV=="Unsure"]


os.chdir(non_expert_chosen_fields_dir + 'NE_Set2_PDFs/')
disagreement_count = 0
discuss_count = 0
comment_count = 0

with open("UnsureAgreements.tex","w") as file:
    file.write("\\documentclass{article}\n")

    file.write("\\usepackage[osf,sc]{mathpazo}\n")
    file.write("\\usepackage{color}\n")
    file.write("\\definecolor{mgreen}{RGB}{25,147,100}\n")

    file.write("\\usepackage{mathtools,hyperref}\n")
    file.write("\\hypersetup{\n")
    file.write("      colorlinks=true,\n")
    file.write("      linkcolor=cyan,\n")
    file.write("      filecolor=cyan,\n")
    file.write("      urlcolor=mgreen,\n")
    file.write("      citecolor=cyan}\n")
    

    file.write("\\usepackage[margin=1in]{geometry}\n")
    file.write("\\usepackage[T1]{fontenc}\n")
    
    file.write("\\usepackage{caption}\n")
    file.write("\\captionsetup{justification= raggedright, singlelinecheck = false}\n")

    file.write("\\title{\\bf Set 2 - Unsure Agreements}\n")
    file.write("\\author{}\n")
    file.write("\\date{}\n")

    file.write("\\begin{document}\n")
    file.write("\\maketitle\n")

    # file.write("\\section{Discussion, Comments, and Disgreements}\n")
    
    file.write("\n")
    file.write("\n")
    
    for a_row in unsureAgreement.index:
        curr_row = unsureAgreement.loc[a_row]
        
        ##### Disagreement
        if len(np.unique(curr_row[["HosseinV", "KirtiV"]].values))>1: # KirtiV
            is_there_disagreement=True
            disagreement_count+=1
        else:
            is_there_disagreement=False
        
        ##### Discussion
        if str(curr_row["HosseinD"])==str(curr_row["KirtiD"])=='No':
            discuss_or_not=False
        else:
            discuss_or_not=True
            discuss_count+=1
        
        ##### Comment
        if str(curr_row["HosseinC"])==str(curr_row["KirtiC"])=='No':
            comment_or_not=False
        else:
            comment_or_not=True
            comment_count+=1
        
        TS_file_name = curr_row["NDVI_TS_Name"]
        file.write("\\begin{figure*}[ht]\n")
        file.write("\\centering\n")
        file.write("\\includegraphics[width=1\\textwidth]{/Users/hn/Documents/" + \
                         "01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/" + \
                             TS_file_name + "}\n")
        file.write("\\caption[]{" + \
                       "\\textbf{\\color{red}{" + curr_row["CropTyp"] + "}}" + \
                       ", ID: " + curr_row["ID"].replace("_", "\_") + \
                       ", Form: " + str(curr_row["Form"]) + \
                       ", Question: " + curr_row["Question"] + "." + \
                       "}\n")
        file.write("\\label{fig:figure" + str(a_row) + "}\n")
        file.write("\\end{figure*}\n")
        file.write("\n")
        
        if discuss_or_not:
            file.write("\\noindent \\textbf{Discuss? {\color{red}{Yes}}}" + "\\\\\n")
        else:
            file.write("\\noindent \\textbf{Discuss? {\color{mgreen}{No}}}" + "\\\\\n")
            
        file.write("\\noindent Kirti: " + str(curr_row["KirtiD"]) + "\\\\\n")
        file.write("\\noindent Hossein: ~" + str(curr_row["HosseinD"]) + "\\\\\n")

        file.write("\n")
        file.write("\n")
        if is_there_disagreement:
            file.write("\\noindent \\textbf{Disagreement? {\color{red}{Yes}}}" + "\\\\\n")
        else:
            file.write("\\noindent \\textbf{Disagreement? {\color{mgreen}{No}}}" + "\\\\\n")
            file.write("\\noindent Kirti's Vote:~~~~ " + str(curr_row["KirtiV"]) + "\\\\\n")
            file.write("Hossein's Vote: " + curr_row['HosseinV'] + "\\\\\n")

            file.write("\n")
            file.write("\n")
            file.write("\\noindent \\textbf{Comments}" + "\\\\\n")
            file.write("\\noindent Kirti: ~~~~" + str(curr_row["KirtiC"]) + "\\\\\n")
            file.write("Hossein: " + str(curr_row["HosseinC"]) + "\\\\\n")
            file.write("\\clearpage\n")

    file.write("\\noindent Disagreement count: " + str(disagreement_count) + "\\\\\n")
    file.write("Discussion count: " + str(discuss_count) + "\\\\\n")
    file.write("Comment count: " + str(comment_count) + "\\\\\n")
    
    file.write("\\end{document}\n")
    
print (disagreement_count)
print (discuss_count)
print (comment_count)
unsureAgreement.shape

0
0
1


(4, 26)

# Label for ML: change damn mustard to double-crop

In [139]:
disagreement_IDs_New = pd.DataFrame(L1+L13, columns = ['disagreement_IDs'])

In [140]:
disagreement_IDs_New.head(2)

,disagreement_IDs
0,106600_WSDA_SF_2018
1,107807_WSDA_SF_2018


In [ ]:
print (all_responses.shape)
print (all_responses.PerryV.unique())
print (all_responses.AndrewV.unique())

In [ ]:
A = all_responses.copy()
A = A[A.PerryV==A.AndrewV]
print (A.PerryV.unique())
print (A.AndrewV.unique())

In [ ]:
double_place= ['Mustard Crop', 'Either double or mustard crop']

all_responses['PerryVCorrected']=all_responses['PerryV']
idx = all_responses[all_responses.PerryVCorrected.isin(double_place)].index
all_responses.loc[idx, "PerryVCorrected"] = 'Double Crop'
all_responses.head(2)

all_responses['AndrewVCorrected']=all_responses['AndrewV']
idx = all_responses[all_responses.AndrewVCorrected.isin(double_place)].index
all_responses.loc[idx, "AndrewVCorrected"] = 'Double Crop'
all_responses.head(2)
pre_meeting_agreements = all_responses[all_responses.PerryVCorrected==
                                                all_responses.AndrewVCorrected].copy()

# create Vote column in pre_meeting_agreements
pre_meeting_agreements['Vote'] = 1
pre_meeting_agreements.head(2)

# Change Vote to 2 for double cropped fields
double_index = pre_meeting_agreements[pre_meeting_agreements.PerryVCorrected=="Double Crop"].index
pre_meeting_agreements.loc[double_index, 'Vote']=2

pre_meeting_agreements = pre_meeting_agreements[["ID", "PerryV", "AndrewV", 
                                                 "PerryVCorrected", "AndrewVCorrected", 
                                                 "Vote"]]

out_folder = "/Users/hn/Documents/01_research_data/NASA/ML_data/"
out_name = out_folder + "expert_set2_premeeting_consensus.csv"
pre_meeting_agreements.to_csv(out_name, index = False)

In [ ]:
all_responses.head(2)

# DO NOT 
include the disagreement of type (double, mustard) in the PDF

In [ ]:
os.chdir('/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/set_2_PDFs/')

disagreement_count = 0

with open("noRepetitions_set_2_3in1_Discussion_Comments_Disagreements.tex","w") as file:
    file.write("\\documentclass{article}\n")

    file.write("\\usepackage[osf,sc]{mathpazo}\n")
    file.write("\\usepackage{color}\n")
    file.write("\\definecolor{mgreen}{RGB}{25,147,100}\n")

    file.write("\\usepackage{mathtools,hyperref}\n")
    file.write("\\hypersetup{\n")
    file.write("      colorlinks=true,\n")
    file.write("      linkcolor=cyan,\n")
    file.write("      filecolor=cyan,\n")
    file.write("      urlcolor=mgreen,\n")
    file.write("      citecolor=cyan}\n")
    

    file.write("\\usepackage[margin=1in]{geometry}\n")
    file.write("\\usepackage[T1]{fontenc}\n")
    
    file.write("\\usepackage{caption}\n")
    file.write("\\captionsetup{justification= raggedright, singlelinecheck = false}\n")

    file.write("\\title{\\bf Set 2 - Discussion, Comments, and Disgreements}\n")
    file.write("\\author{}\n")
    file.write("\\date{}\n")


    file.write("\\begin{document}\n")
    file.write("\\maketitle\n")

    # file.write("\\section{Discussion, Comments, and Disgreements}\n")
    
    file.write("\n")
    file.write("\n")
    
    for a_row in all_responses.index:
        curr_row = all_responses.loc[a_row]
        
        if len(np.unique(curr_row[["PerryVCorrected", "AndrewVCorrected"]].values))>1:
            is_there_disagreement=True
            disagreement_count+=1
        else:
            is_there_disagreement=False
        
        if str(curr_row["PerryD"])==str(curr_row["AndrewD"])=='No':
            discuss_or_not=False
        else:
            discuss_or_not=True
        
        if str(curr_row["PerryC"])==str(curr_row["AndrewC"])=='No':
            comment_or_not=False
        else:
            comment_or_not=True
        
        if is_there_disagreement or discuss_or_not or comment_or_not:
            TS_file_name = curr_row["NDVI_TS_Name"]
            file.write("\\begin{figure*}[ht]\n")
            file.write("\\centering\n")
            file.write("\\includegraphics[width=1\\textwidth]{/Users/hn/Documents/" + \
                         "01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/" + \
                             TS_file_name + "}\n")
            file.write("\\caption[]{" + \
                       "\\textbf{\\color{red}{" + curr_row["CropTyp"] + "}}" + \
                       ", ID: " + curr_row["ID"].replace("_", "\_") + \
                       ", Form: " + curr_row["Form"] + \
                       ", Question: " + curr_row["Question"] + "." + \
                       "}\n")
            file.write("\\label{fig:figure" + str(a_row) + "}\n")
            file.write("\\end{figure*}\n")
            file.write("\n")
        
            if discuss_or_not:
                file.write("\\noindent \\textbf{Discuss? {\color{red}{Yes}}}" + "\\\\\n")
            else:
                file.write("\\noindent \\textbf{Discuss? {\color{mgreen}{No}}}" + "\\\\\n")
            
            file.write("\\noindent Perry: ~~~~" + str(curr_row["PerryD"]) + "\\\\\n")
            file.write("Andrew: " + str(curr_row["AndrewD"]) + "\\\\\n")

            file.write("\n")
            file.write("\n")
            if is_there_disagreement:
                file.write("\\noindent \\textbf{Disagreement? {\color{red}{Yes}}}" + "\\\\\n")
                file.write("\\noindent Perry's Vote:~~~~ " + str(curr_row["PerryV"]) + "\\\\\n")
                file.write("Andrew's Vote: " + str(curr_row["AndrewV"]) + "\\\\\n")
            else:
                file.write("\\noindent \\textbf{Disagreement? {\color{mgreen}{No}}}" + "\\\\\n")
                file.write("\\noindent Perry's Vote:~~~~ " + curr_row['PerryV'] + "\\\\\n")
                file.write("Andrew's Vote: " + curr_row['AndrewV'] + "\\\\\n")

            file.write("\n")
            file.write("\n")
            file.write("\\noindent \\textbf{Comments}" + "\\\\\n")
            file.write("\\noindent Perry: ~~~~" + str(curr_row["PerryC"]) + "\\\\\n")
            file.write("Andrew: " + str(curr_row["AndrewC"]) + "\\\\\n")

            file.write("\\clearpage\n")

    file.write("disagreement count: " + str(disagreement_count) + "\\\\\n")
    file.write("\\end{document}\n")